# Figure out the optimal hyperparameter settings for the demixing process

In [18]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib as mpl
import time
import matplotlib.pyplot as plt
from matplotlib.path import Path as mplPath
import matplotlib.cm as cm
import matplotlib.patches as patches
from scipy import stats
import math
import numpy as np
from scipy import interpolate
from pathlib import Path
import os 

from gating_util import ScatterSelectorGating

from bsccm import BSCCM
from demixing_util import *

bsccm = BSCCM(str(Path.home()) + '/BSCCM_local/BSCCM/')
dataframe_saving_fullpath = str(Path.home()) + '/BSCCM_local/BSCCM/BSCCM_surface_markers.csv'
# For exporting figures
export_dir = '/home/henry/leukosight_data/figures/demixing'


single_markers = ['CD123', 'CD3', 'CD19', 'CD56', 'HLA-DR', 'CD45', 'CD14', 'CD16', 'autofluor']

# The names of the fluorescent measurements put into a database
channel_names = ['Fluor_426-446_shading_corrected', 
       'Fluor_500-550_shading_corrected', 
       'Fluor_550-570_shading_corrected', 
       'Fluor_585-625_shading_corrected', 
       'Fluor_627-673_shading_corrected', 
       'Fluor_690-_shading_corrected']

selections = {batch: ['selection_example_{}_positive_cells_batch_{}'.format(m, batch) 
               for m in single_markers] for batch in range(2)}

unmixed_channel_names = ['CD123/HLA-DR/CD14', 'CD3/CD19/CD56', 'CD45', 'CD16', 'autofluor']


single_marker_unmix_channel_spectra, single_marker_unmix_channel_brightness, unmix_channel_spectra, unmix_channel_brightness = \
        compute_spectra(bsccm, channel_names, unmixed_channel_names, single_markers, batch=0)
    
# Prepare the raw data
mixed_data = bsccm.surface_marker_dataframe[channel_names].to_numpy()
#Make everything positive
mixed_data -= (np.min(mixed_data, axis=0) - 1e-2)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Experiment to look at tradeoff between regularization level on single marker + autofluor

In [13]:
scatter_fig, scatter_ax = plt.subplots(1, 2, figsize=(7, 2.5))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
experiment_path = '/home/henry/leukosight_data/demixing_results/single_marker/'

for single_marker in single_markers:
    if single_marker == 'autofluor':
        continue #no need to do this when only two markers
    #get the unmixed channel that contains this marker
#     unmixed_channel_name = [m for m in unmix_channel_spectra if single_marker in m][0]
#     spectra = np.stack([unmix_channel_spectra[c] for c in [unmixed_channel_name, 'autofluor']], axis=0)
    spectra_names = [single_marker, 'autofluor']
    spectra = np.stack([single_marker_unmix_channel_spectra[c] for c in spectra_names], axis=0)
    reweighting = [1, 1]

    params = [
        (single_marker, 1e-2, reweighting),
        (single_marker, 3e-1, reweighting),
        (single_marker, 1e-1, reweighting),
        (single_marker, 1e0, reweighting),
        (single_marker, 3e0, reweighting),
        (single_marker, 1e1, reweighting),
    ]

    for stain_antibodies, l1_reg, reweighting in params:
        print('Running {} {} {}\t\t\t\t\t\t\t\t\t'.format(stain_antibodies, l1_reg, reweighting))
        run_experiment(spectra, spectra_names, bsccm, mixed_data, stain_antibodies, selections, reweighting, l1_reg, 
                       scatter_fig, scatter_ax,  path = experiment_path, callback_every=500)


Running CD123 0.01 [1, 1]									
Running CD123 0.3 [1, 1]									05		3.4  6.6  3.0  7.4  3.8  15.1  					


KeyboardInterrupt: 

In [17]:
show_experiment_results('/home/henry/leukosight_data/demixing_results/single_marker/')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Experiment to look at reweighting of two marker spectra

In [48]:
scatter_fig, scatter_ax = plt.subplots(1, 2, figsize=(7, 2.5))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
experiment_path = '/home/henry/leukosight_data/demixing_results/single_marker_reweighted_spectra/'

for single_marker in single_markers:
    if single_marker == 'autofluor':
        continue #no need to do this when only two markers
    #get the unmixed channel that contains this marker
#     unmixed_channel_name = [m for m in unmix_channel_spectra if single_marker in m][0]
#     spectra = np.stack([unmix_channel_spectra[c] for c in [unmixed_channel_name, 'autofluor']], axis=0)
    spectra_names = [single_marker, 'autofluor']
    spectra = np.stack([single_marker_unmix_channel_spectra[c] for c in spectra_names], axis=0)
#     reweighting = [1, 1]

    params = [
        (single_marker, 1e1, [1, 1]),
        (single_marker, 1e1, 
         [single_marker_unmix_channel_brightness[single_marker] / single_marker_unmix_channel_brightness['autofluor'], 1] ),
        (single_marker, 1e1, 
         [single_marker_unmix_channel_brightness['autofluor'] / single_marker_unmix_channel_brightness[single_marker], 1]),
    ]

    for stain_antibodies, l1_reg, reweighting in params:
        print('Running {} {} {}\t\t\t\t\t\t\t\t\t'.format(stain_antibodies, l1_reg, reweighting))
        run_experiment(spectra, spectra_names, bsccm, mixed_data, stain_antibodies, selections, reweighting, l1_reg, 
                       scatter_fig, scatter_ax,  path = experiment_path, callback_every=500)


Running CD123 10.0 [1, 1]									
Running CD123 10.0 [1.7420103374497333, 1]									4.7  22.8  14.5  14.9  				
Running CD123 10.0 [0.5740494063106302, 1]									4.7  22.8  14.5  14.9  				
Running CD3 10.0 [1, 1]									0.0005		16.9  40.9  4.1  21.4  13.5  15.1  				
Running CD3 10.0 [4.966696429093412, 1]									7  6.2  25.3  16.0  15.5  				
Running CD3 10.0 [0.2013410753559048, 1]									  6.5  27.5  17.9  15.4  				
Running CD19 10.0 [1, 1]									0005		36.3  6.6  -0.0  -0.0  -0.0  12.1  				
Running CD19 10.0 [5.4125314696399185, 1]									 4.2  20.7  13.2  14.8  				
Running CD19 10.0 [0.18475643155318733, 1]									4.1  21.0  13.6  14.7  				
Running CD56 10.0 [1, 1]									0005		33.3  -0.0  -0.0  -0.0  -0.0  12.7  				
Running CD56 10.0 [3.4396871100158455, 1]									 6.8  29.0  18.8  16.3  				
Running CD56 10.0 [0.2907241176350466, 1]									 6.8  29.4  19.3  16.3  				
Running HLA-DR 10.0 [1, 1]									05		36.8  9.9  -0.0  -0.0  -0.0  11.3  				
Running HLA-DR 

In [51]:
show_experiment_results('/home/henry/leukosight_data/demixing_results/single_marker_reweighted_spectra/')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Experiment to look at projection reweighting of two marker spectra

In [3]:
scatter_fig, scatter_ax = plt.subplots(1, 2, figsize=(7, 2.5))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
experiment_path = '/home/henry/leukosight_data/demixing_results/single_marker_projection_reweighted_spectra/'

for single_marker in single_markers:
    if single_marker == 'autofluor':
        continue #no need to do this when only two markers
    #get the unmixed channel that contains this marker
#     unmixed_channel_name = [m for m in unmix_channel_spectra if single_marker in m][0]
#     spectra = np.stack([unmix_channel_spectra[c] for c in [unmixed_channel_name, 'autofluor']], axis=0)
    spectra_names = [single_marker, 'autofluor']
    spectra = np.stack([single_marker_unmix_channel_spectra[c] for c in spectra_names], axis=0)
#     reweighting = [1, 1]

    autofluor_mag = single_marker_unmix_channel_brightness['autofluor']
    marker_mag = single_marker_unmix_channel_brightness[single_marker]
    autofluor_vec = single_marker_unmix_channel_spectra['autofluor']
    marker_vec = single_marker_unmix_channel_spectra[single_marker]
    weighted_proj = (autofluor_vec @ marker_vec) / marker_mag
    params = [
        (single_marker, 7e-1, [weighted_proj, 1] ),
    ]

    for stain_antibodies, l1_reg, reweighting in params:
        print('Running {} {} {}\t\t\t\t\t\t\t\t\t'.format(stain_antibodies, l1_reg, reweighting))
        run_experiment(spectra, spectra_names, bsccm, mixed_data, stain_antibodies, selections, reweighting, l1_reg, 
                       scatter_fig, scatter_ax,  path = experiment_path, callback_every=5000)


Running CD123 1.0 [0.005845640062146342, 1]									
Running CD123 2.0 [0.005845640062146342, 1]									.1  18.4  12.0  17.6  				
Running CD3 1.0 [0.008726980330218712, 1]									 6.3  20.1  13.2  17.6  				
Running CD3 2.0 [0.008726980330218712, 1]									2.6  -0.0  -0.0  16.5  				
Running CD19 1.0 [0.00793725786794061, 1]									1.8  -0.0  -0.0  15.9  				
Running CD19 2.0 [0.00793725786794061, 1]									1.9  0.9  1.0  17.0  				
Running CD56 1.0 [0.011405274445969964, 1]									.4  0.2  0.4  16.4  				
Running CD56 2.0 [0.011405274445969964, 1]									.2  1.1  -0.0  16.0  					
Running HLA-DR 1.0 [0.005821131558984232, 1]									  -0.0  -0.0  15.4  				
Running HLA-DR 2.0 [0.005821131558984232, 1]									  8.9  4.9  14.7  				
Running CD45 1.0 [0.0028867382637343393, 1]									5  11.2  6.5  14.8  				
Running CD45 2.0 [0.0028867382637343393, 1]									.3  20.4  4.6  6.2  				
Running CD14 1.0 [0.015182378836181449, 1]									8.1  20.9  4.5  5.6  				
Running CD14 2.0 [

In [36]:
show_experiment_results('/home/henry/leukosight_data/demixing_results/single_marker_projection_reweighted_spectra/')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/henry/leukosight_data/demixing_results/single_marker_projection_reweighted_spectra/HLA-DR__0.7__0.01-1.00.png


### Experiment for single marker spectra, all_marker model

In [21]:
scatter_fig, scatter_ax = plt.subplots(1, 5, figsize=(15, 2.5))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
experiment_path = '/home/henry/leukosight_data/demixing_results/single_marker_data_all_marker_model/'

#compute weighting based on projection onto first singular vector
unmix_spectrum = np.array([unmix_channel_spectra[name] * unmix_channel_brightness[name] for name in unmixed_channel_names])
u, s, vh = np.linalg.svd(unmix_spectrum, full_matrices=False)
first_vec = np.abs(vh[0])
weightings = [first_vec @ unmix_channel_spectra[name] / unmix_channel_brightness[name]
         for name in unmixed_channel_names]

for single_marker in single_markers:
    spectra_names = unmixed_channel_names
    spectra = np.stack([unmix_channel_spectra[c] for c in spectra_names], axis=0)

    params = [
#         (single_marker, 1e-2, weightings),
#         (single_marker, 1e-1, weightings),
        (single_marker, 1e0, weightings),
         (single_marker, 1e2, weightings),
    ]

    for stain_antibodies, l1_reg, reweighting in params:
        print('Running {} {} {}\t\t\t\t\t\t\t\t\t'.format(stain_antibodies, l1_reg, reweighting))
        run_experiment(spectra, spectra_names, bsccm, mixed_data, stain_antibodies, selections, reweighting, l1_reg, 
                       scatter_fig, scatter_ax,  path = experiment_path, callback_every=5000)


Running CD123 1.0 [0.017437714405862902, 0.0032108197145691614, 0.005827361691958369, 0.0034276290867762474, 0.017140592104324053]									
Running CD123 100.0 [0.017437714405862902, 0.0032108197145691614, 0.005827361691958369, 0.0034276290867762474, 0.017140592104324053]									
Running CD3 1.0 [0.017437714405862902, 0.0032108197145691614, 0.005827361691958369, 0.0034276290867762474, 0.017140592104324053]									
Running CD3 100.0 [0.017437714405862902, 0.0032108197145691614, 0.005827361691958369, 0.0034276290867762474, 0.017140592104324053]									
Running CD19 1.0 [0.017437714405862902, 0.0032108197145691614, 0.005827361691958369, 0.0034276290867762474, 0.017140592104324053]									
Running CD19 100.0 [0.017437714405862902, 0.0032108197145691614, 0.005827361691958369, 0.0034276290867762474, 0.017140592104324053]									
Running CD56 1.0 [0.017437714405862902, 0.0032108197145691614, 0.005827361691958369, 0.0034276290867762474, 0.017140592104324053]									
Running CD56 100.0 [0

In [24]:
show_experiment_results('/home/henry/leukosight_data/demixing_results/single_marker_data_all_marker_model/')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/henry/leukosight_data/demixing_results/single_marker_data_all_marker_model/HLA-DR__0.1__0.02-0.00-0.01-0.00-0.02.png


### Experiment for single marker spectra, all_marker model (No autofluor)

In [38]:
scatter_fig, scatter_ax = plt.subplots(1, 4, figsize=(15, 2.5))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
experiment_path = '/home/henry/leukosight_data/demixing_results/single_marker_data_all_marker_model_no_af/'

#compute weighting based on projection onto first singular vector
unmix_spectrum = np.array([unmix_channel_spectra[name] * unmix_channel_brightness[name] for name in unmixed_channel_names])
u, s, vh = np.linalg.svd(unmix_spectrum, full_matrices=False)
first_vec = np.abs(vh[0])
weightings = [first_vec @ unmix_channel_spectra[name] / unmix_channel_brightness[name]
         for name in unmixed_channel_names]

for single_marker in single_markers[:-1]:
    spectra_names = unmixed_channel_names[:-1]
    spectra = np.stack([unmix_channel_spectra[c] for c in spectra_names], axis=0)

    params = [
#         (single_marker, 1e-2, weightings),
#         (single_marker, 1e-1, weightings),
        (single_marker, 7e-1, weightings),
#          (single_marker, 1e2, weightings),
    ]

    for stain_antibodies, l1_reg, reweighting in params:
        print('Running {} {} {}\t\t\t\t\t\t\t\t\t'.format(stain_antibodies, l1_reg, reweighting))
        run_experiment(spectra, spectra_names, bsccm, mixed_data, stain_antibodies, selections, reweighting, l1_reg, 
                       scatter_fig, scatter_ax,  path = experiment_path, callback_every=5000)

Running CD123 0.7 [0.017437714405862902, 0.0032108197145691614, 0.005827361691958369, 0.0034276290867762474, 0.017140592104324053]									
Running CD3 0.7 [0.017437714405862902, 0.0032108197145691614, 0.005827361691958369, 0.0034276290867762474, 0.017140592104324053]									
Running CD19 0.7 [0.017437714405862902, 0.0032108197145691614, 0.005827361691958369, 0.0034276290867762474, 0.017140592104324053]									
Running CD56 0.7 [0.017437714405862902, 0.0032108197145691614, 0.005827361691958369, 0.0034276290867762474, 0.017140592104324053]									
Running HLA-DR 0.7 [0.017437714405862902, 0.0032108197145691614, 0.005827361691958369, 0.0034276290867762474, 0.017140592104324053]									
Running CD45 0.7 [0.017437714405862902, 0.0032108197145691614, 0.005827361691958369, 0.0034276290867762474, 0.017140592104324053]									
Running CD14 0.7 [0.017437714405862902, 0.0032108197145691614, 0.005827361691958369, 0.0034276290867762474, 0.017140592104324053]									
Running CD16 0.7 [0.01743

In [35]:
show_experiment_results('/home/henry/leukosight_data/demixing_results/single_marker_data_all_marker_model_no_af/')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/henry/leukosight_data/demixing_results/single_marker_data_all_marker_model_no_af/HLA-DR__0.7__0.02-0.00-0.01-0.00-0.02.png


In [34]:
single_markers

['CD123', 'CD3', 'CD19', 'CD56', 'HLA-DR', 'CD45', 'CD14', 'CD16', 'autofluor']